In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

# Stage 2

## Task 1

Compare the weekly statistics (Mean, Median, Mode) for cases and deaths across the US.

In [58]:
super_covid = pd.read_csv('./data/super_covid_data.csv')
covid_cases = pd.read_csv('./data/covid_confirmed_usafacts.csv')
covid_deaths = pd.read_csv('./data/covid_deaths_usafacts.csv')

# It will be easier to handle these separately so I'm importing the deaths and cases CSVs again.

# Drop Columns from COVID death statistics to prep transpose
covid_deaths.drop(covid_deaths.columns[[0,1,2,3]], axis=1, inplace=True)

# Transfer that information to a dataframe I want to manipulate
cd_date = covid_deaths

# Re-read the original data into the covid deaths dataframe because for some reason I can't get it to work any other way.
covid_deaths = pd.read_csv('./data/covid_deaths_usafacts.csv')

#Transpose
cd_date = cd_date.transpose()

# Rename the columns after the countyFIPS so that I can merge later.
cd_date.columns = covid_deaths['countyFIPS']

# Change the index to a date and time so that I can resample it.
cd_date.index = pd.to_datetime(cd_date.index)

#Repeat this process for Cases

# Resample and Assess
death_sums = cd_date.resample('W').sum()
death_mean = cd_date.resample('W').mean()

death_sums

countyFIPS,0,1001,1003,1005,1007,1009,1011,1013,1015,1017,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-25,0,1645,5117,728,777,1827,378,924,4781,1204,...,84,1078,322,644,196,987,112,301,357,168
2023-07-02,0,1645,5117,728,777,1827,378,924,4781,1204,...,84,1078,322,644,196,991,112,301,357,168
2023-07-09,0,1645,5117,728,777,1827,378,924,4781,1204,...,84,1078,322,644,196,994,112,301,357,168
2023-07-16,0,1645,5117,728,777,1827,378,924,4781,1204,...,84,1078,322,644,196,994,112,301,357,168
